In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
from WindPy import w
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

## Mysql 封装

In [3]:
from sqlalchemy import create_engine
import MySQLdb

class Mysql_helper():

    def __init__(self, database='option'):
        self.engine = create_engine('mysql://root:@localhost/{}'.format(database))
        self.db = MySQLdb.connect('localhost', 'root', '', database)
        self.cur = self.db.cursor()
        
    def insert(self, data, table_name,if_exists='append'):
        data.to_sql(table_name, self.engine, if_exists=if_exists, index=False, chunksize=1000)
    
    def read(self, table_name):
        data = pd.read_sql(table_name, self.engine)
        return data
    
    def query(self, sql_query):
        data = pd.read_sql_query(sql_query, self.engine)
        return data
    
    
    def sql(self, sql):        
        try:
            self.cursor.execute(sql)
            self.db.commit()
        except:
            # Rollback in case there is any error
            self.db.rollback()

## Mongodb 封装

In [4]:
from pymongo import MongoClient

class Mongo_helper():    
    def __init__(self, database='option', url='localhost:27017', table=None):
        self.client = MongoClient('mongodb://{}'.format(url))
        self.database = self.client[database]
        if table is not None:
            self.table = self.client[database][table] 
        else:
            self.table = self.database[self.database.collection_names()[0]]
        
    def insert(self, data):
        if type(data) == pd.core.frame.DataFrame:
            self.table.insert_many(data.to_dict('records'))
        else:
            self.table.insert_many(data)
    
    def read_df(self,):
        df = pd.DataFrame(list(self.table.find()))
        return df.drop('_id', axis=1) if df.shape[0]!=0 else None
    
    def query(self, conditions):
        df = pd.DataFrame(list(self.table.find(eval(conditions))))
        return df.drop('_id', axis=1) if df.shape[0]!=0 else None
    
    def update(self, conditions, updata):
        self.table.update_many(eval(conditions),{"$set":eval(updata)})
    
    def delete(self, conditions):
        self.table.delete_many(eval(conditions))
        


## 获得期权数据函数

参数字段：期权代码，日期，开始日期，结束日期，行权价，类型，隐含波动率，开盘价,收盘价，最高价，最低价，交易量，持仓量

In [5]:
def get_volatility(code_dir):
    
    """
    根据每个期权合约代码得到期权数据
    输入：code - 期权合约代码
    输出：data - dataframe 形式存储
    
    参考字段：
    'TRADE_CODE', 'US_NAME', 'LASTRADEDAY_S', 'STARTDATE', 'EXE_ENDDATE','EXE_PRICE', 'EXE_MODE',
    'US_IMPLIEDVOL', 'PTMTRADEDAY', 'OPEN', 'HIGH','LOW', 'CLOSE', 'VOLUME', 'OI', 'US_CLOSE'

    期权代码，标的代码，最近交易日，合约起始日，合约终止日，期权价格， 期权类型，
    隐含波动率，到期时间，开盘价，最高价，最低价，收盘价，成交量，持仓量，标的价格
    """
    from WindPy import w    
    w.start()
    
    date_range = w.wsd(code_dir, "startdate, exe_enddate,", "2015-03-23", "2015-03-23",)
    fields = "trade_code,us_name,lastradeday_s,startdate,exe_enddate,exe_price,exe_mode,us_impliedvol,ptmtradeday,open,high,low,close,volume,oi, us_close"
    data  = w.wsd(code_dir, fields, date_range.Data[0][0], date_range.Data[1][0])
    df = pd.DataFrame(data.Data, index=data.Fields).T
    df['EXE_MODE'] = df['EXE_MODE'].map({'认购': 'call', '认沽': 'put'})
    df.dropna(inplace=True)
    
    return df

## 下载所有数据并保存到数据库

In [6]:
def save_all_volatility_to_mongo(code_list):
    mgh = Mongo_helper(database='option_data', table='ETF50')
    for code_dir in code_list:     
        try:            
            data = get_volatility(code_dir)
            mgh.insert(data.to_dict('records'))
            print('Succeed download: ', code_dir)
        except:
            print('Error: ', code_dir)

In [7]:
def update_volatility_to_mongo(code_list):
    mgh = Mongo_helper(database='option_data', table='ETF50')
    for code_dir in code_list:     
        try:            
            data = get_volatility(code_dir)
            [mgh.table.update_many({'LASTRADEDAY_S': data['LASTRADEDAY_S'][i], 
                                    'EXE_ENDDATE': data['EXE_ENDDATE'][i]},
                                   {"$set":data.iloc[i].to_dict()}, upsert=True) for i in range(len(data))]
            print('Succeed download: ', code_dir)
        except:
            print('Error: ', code_dir)

In [8]:
## update data everyday
def update_mongo_database(database):
    mongo = Mongo_helper(database=database)
    min_code = pd.DataFrame(list(mongo.table.find({'EXE_ENDDATE':{'$gte':pd.datetime.today()}}, {'TRADE_CODE':''})))['TRADE_CODE'].min()

    code = int(min_code)
    while True:    
        df = get_volatility(str(code) + '.SH')
        if df.empty:
            break
        else:
            mongo.table.delete_many({'TRADE_CODE': str(code)})
            mongo.insert(df)
            print('Succeed Update:', code)
            code += 1 